In [129]:
# ! pip install transformers
import torch
import numpy as np
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW

from torch import nn
from transformers import BertModel
from transformers import BertTokenizer

from sklearn.model_selection import train_test_split
import torch

import pandas as pd
import numpy as np
import os

In [130]:
# df[df['content']==' ']

In [131]:
torch.cuda.empty_cache()

In [132]:

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/')


df=pd.read_csv('BERT_data.csv')
df=df[~(df['content']=='nan')]
df['content']=df['content'].astype(str)
df['subject']=df['subject'].astype(str)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [133]:
'''
df=pd.read_csv('BERT_data.csv')
df['content']=df['content'].astype(str)
df['subject']=df['subject'].astype(str)
'''

"\ndf=pd.read_csv('BERT_data.csv')\ndf['content']=df['content'].astype(str)\ndf['subject']=df['subject'].astype(str)\n"

In [134]:
df['cat_1_level_2']=df['cat_1_level_2']-1
df['cat_1_level_2'].unique()

array([0, 2, 3, 4, 1, 6, 5, 7])

In [135]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
class Dataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.labels=df['cat_1_level_2']
        self.text=[tokenizer(text,
                               padding='max_length',truncation=True,
                                return_tensors="pt") for text in df['content']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.text[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [136]:

df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.6*len(df)), int(.8*len(df))])


In [137]:
from sklearn.utils import resample
for i in [1,2,3,4,5,6]:
    spam_upsample = resample(df_train[df_train['cat_1_level_2']==i],
             replace=True,
             n_samples=len(df_train[df_train['cat_1_level_2']==0]) - len(df_train[df_train['cat_1_level_2']==i]),
             random_state=42)
    df_train=pd.concat([df_train,spam_upsample])
df_train.shape

(3016, 4)

In [138]:
df_train.reset_index(inplace=True)
df_train=df_train.drop('index',axis=1)

df_val.reset_index(inplace=True)
df_val=df_val.drop('index',axis=1)

df_test.reset_index(inplace=True)
df_test=df_test.drop('index',axis=1)

print(len(df_train),len(df_val), len(df_test))

3016 298 298


In [139]:
'''
model_name = "bert-base-uncased"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=8)
'''

'\nmodel_name = "bert-base-uncased"\nmodel = BertForSequenceClassification.from_pretrained(model_name, num_labels=8)\n'

In [140]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.3):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 8)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer


In [141]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [142]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=65)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=65)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0
            n=0
            for train_input, train_label in tqdm(train_dataloader):
                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)


                output = model(input_id, mask)
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')

EPOCHS = 100
model = BertClassifier()
LR = 0.000001

train(model, df_train, df_val, LR, EPOCHS)

100%|██████████| 47/47 [00:59<00:00,  1.27s/it]


Epochs: 1 | Train Loss:  0.034                 | Train Accuracy:  0.114                 | Val Loss:  0.034                 | Val Accuracy:  0.211


100%|██████████| 47/47 [00:59<00:00,  1.27s/it]


Epochs: 2 | Train Loss:  0.033                 | Train Accuracy:  0.139                 | Val Loss:  0.034                 | Val Accuracy:  0.181


100%|██████████| 47/47 [00:59<00:00,  1.27s/it]


...

Epochs: 20 | Train Loss:  0.024                 | Train Accuracy:  0.536                 | Val Loss:  0.034                 | Val Accuracy:  0.218


100%|██████████| 47/47 [00:59<00:00,  1.27s/it]


Epochs: 21 | Train Loss:  0.024                 | Train Accuracy:  0.559                 | Val Loss:  0.034                 | Val Accuracy:  0.208


 51%|█████     | 24/47 [00:32<00:30,  1.34s/it]


KeyboardInterrupt: ignored

In [143]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

evaluate(model, df_test)

Test Accuracy:  0.282
